In [68]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime


In [94]:
def habr_search_to_dataframe(queries):
    articles = []
    seen_urls = set()

    
    
    for query in queries:
        url = f'https://habr.com/ru/search/?q={query}&target_type=posts'
        response = requests.get(url)
        
        if response.status_code != 200:
            continue
            
        soup = BeautifulSoup(response.text)
        
        for article in soup.find_all('article', class_='tm-articles-list__item'):
            title_elem = article.find('a', class_='tm-title__link')
            time_elem = article.find('time')
            
            if not title_elem or not time_elem:
                continue
                
            link = 'https://habr.com' + title_elem['href']
            if link in seen_urls:
                continue
                
            seen_urls.add(link)
            
            date_str = time_elem['datetime']
            date_obj = datetime.fromisoformat(date_str)
            
            articles.append({
                'Дата': date_obj.date(),
                'Заголовок': title_elem.text.strip(),
                'Ссылка': link
            })
    
    return pd.DataFrame(articles).drop_duplicates('Ссылка')



In [100]:
df = habr_search_to_dataframe(['python', 'анализ данных'])
df.head(10)

,Дата,Заголовок,Ссылка
0,2025-02-13,В рейтинге TIOBE начали расти «быстрые» языки ...,https://habr.com/ru/news/882048/
1,2022-01-20,Курс «Python для инженеров». Старт 3 потока 31...,https://habr.com/ru/companies/slurm/news/646825/
2,2024-12-24,Один из ключевых разработчиков CPython Тим Пит...,https://habr.com/ru/news/869196/
3,2024-09-02,55% Python-разработчиков используют Linux-окру...,https://habr.com/ru/news/840224/
4,2021-12-13,Жаждущим автоматизации: открытый урок «ChatOps...,https://habr.com/ru/companies/slurm/news/595093/
5,2022-01-13,Открытый урок «Пишем Custom Prometheus Exporte...,https://habr.com/ru/companies/slurm/news/645485/
6,2023-09-19,Microsoft представила дополнение Python Editor...,https://habr.com/ru/news/761862/
7,2020-04-21,"Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/498364/
8,2023-08-22,Microsoft добавила Python в Excel,https://habr.com/ru/news/756266/
9,2023-11-09,Alchemer совместно с JetBrains и Python Softwa...,https://habr.com/ru/news/772872/


In [ ]:
def habr_advanced_parser(queries, pages=1):
    articles = []
    seen_urls = set()
    
    
    with requests.Session() as session:
        for query in queries:
            for page in range(1, pages+1):
                url = f'https://habr.com/ru/search/page{page}/?q={query}&target_type=posts'
                try:
                    response = session.get(url)
                    response.raise_for_status()
                    soup = BeautifulSoup(response.text, 'html.parser')
                    
                    for article in soup.find_all('article', class_='tm-articles-list__item'):
                        title_elem = article.find('a', class_='tm-title__link')
                        time_elem = article.find('time')
                        
                        if not title_elem or not time_elem:
                            continue
                            
                        link = 'https://habr.com' + title_elem['href']
                        if link in seen_urls:
                            continue
                            
                        seen_urls.add(link)
                        
                        # Парсинг детальной страницы
                        article_response = session.get(link, headers=headers)
                        if article_response.status_code != 200:
                            continue
                            
                        article_soup = BeautifulSoup(article_response.text, 'html.parser')
                        
                        # Извлечение полного текста
                        content = article_soup.find('div', class_='article-formatted-body')
                        full_text = content.get_text(separator='\n').strip() if content else ''
                        
                        # Извлечение лайков
                        votes = article_soup.find('span', class_='tm-votes-meter__value')
                        likes = int(votes.text.strip()) if votes else 0
                        
                        articles.append({
                            'Дата': datetime.fromisoformat(time_elem['datetime']).date(),
                            'Заголовок': title_elem.text.strip(),
                            'Ссылка': link,
                            'Текст': full_text,
                            'Лайки': likes
                        })
                        time.sleep(1)  # Задержка для снижения нагрузки
                        
                except requests.exceptions.RequestException:
                    continue
                time.sleep(2)  # Пауза между страницами
    
    return pd.DataFrame(articles).drop_duplicates('Ссылка')


In [ ]:
df = habr_advanced_parser(['python', 'анализ данных'], pages=3)
print(f"Найдено статей: {len(df)}")
print(df[['Дата', 'Лайки', 'Заголовок']].head())